# Lemmatize file with Perseids Morpheus Parser

## Access Perseids Morpheus and query Lemmata and Morphology

In [ ]:
import sys
import requests # query website
import json
from nltk.tokenize import RegexpTokenizer # tokenize
def queryMorpheusLem(word):
    """Works only online. Query Perseids Morpeus for Lemma (code inspired by ideas from CLTK's lemma.py)."""
    suche = {"word": word}
    try:
        r = requests.get('http://morph.perseids.org/analysis/word?lang=grc&engine=morpheusgrc', data=suche, timeout=10)
    except requests.exceptions.Timeout:
        print("Timeout. Please try again.")
    else:
        data = r.json()
        result = {} # output results as dictionary
        result["Word"] = word
        try:
            entry = data["RDF"]["Annotation"]["Body"]
        except KeyError:
            result["Lemma"] = word # Copy word as lemma 
            result["Morphology"] = "NOT FOUND"
            result["Error"] = "NOT FOUND"  # error indicator
        else:
            # Test if there are more entries
            # TODO: Better way to deal with words with more lemma entries: how to find the correct lemma?
            try: 
                entry[1]["rest"]
                list_hdwd = []
                list_pos = []
                for i in range(len(entry)):
                    headword = entry[i]["rest"]["entry"]["dict"]["hdwd"]["$"]
                    infl = entry[i]["rest"]["entry"]["infl"]
                    if isinstance(infl, list):
                        list_pos1 = []
                        for inst in range(len(infl)):
                            POS = infl[inst]["pofs"]["$"]
                            if POS == "verb":
                                MOOD = infl[inst]["mood"]["$"]
                                TENSE = infl[inst]["tense"]["$"]
                                VOICE = infl[inst]["voice"]["$"]
                                try:
                                    NUM = infl[inst]["num"]["$"]
                                    PERS = infl[inst]["pers"]["$"]
                                    posentry = POS+"-"+MOOD+"-"+NUM+"-"+PERS+"-"+TENSE+"-"+VOICE
                                except KeyError:
                                    posentry = POS+"-"+MOOD+"-"+TENSE+"-"+VOICE
                            elif POS == "verb participle":
                                CASE = infl[inst]["case"]["$"]
                                GEND = infl[inst]["gend"]["$"]
                                NUM = infl[inst]["num"]["$"]                                
                                TENSE = infl[inst]["tense"]["$"]
                                VOICE = infl[inst]["voice"]["$"]  
                                posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM+"-"+TENSE+"-"+VOICE
                            elif POS == "pronoun":
                                CASE = infl[inst]["case"]["$"]
                                NUM = infl[inst]["num"]["$"]
                                posentry = POS+"-"+CASE+"-"+NUM
                            elif POS == "noun":
                                try:
                                    CASE = infl[inst]["case"]["$"]
                                    GEND = infl[inst]["gend"]["$"]
                                    NUM = infl[inst]["num"]["$"]
                                    posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                                except KeyError:
                                    TYPE = infl[inst]["stemtype"]["$"]
                                    posentry = POS+"-"+TYPE
                            elif POS == "article":
                                CASE = infl[inst]["case"]["$"]
                                GEND = infl[inst]["gend"]["$"]
                                NUM = infl[inst]["num"]["$"]
                                posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                            elif POS == "adjective":
                                try:
                                    GEND = infl[inst]["gend"]["$"]
                                    if GEND == "adverbial":
                                        posentry = POS+"-"+GEND
                                    else:
                                        CASE = infl[inst]["case"]["$"]
                                        NUM = infl[inst]["num"]["$"]
                                        try:
                                            COMP = infl[inst]["comp"]["$"]
                                            posentry = POS+"-"+CASE+"-"+GEND+"-"+COMP+"-"+NUM
                                        except KeyError:
                                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                                except KeyError:
                                    CASE = infl[inst]["case"]["$"]
                                    NUM = infl[inst]["num"]["$"]
                                    try:
                                        COMP = infl[inst]["comp"]["$"]
                                        posentry = POS+"-"+CASE+"-"+COMP+"-"+NUM
                                    except KeyError:
                                        posentry = POS+"-"+CASE+"-"+NUM
                            else:
                                posentry = POS
                            list_pos1.append(posentry)
                        instance = "/".join(list_pos1)
                        list_pos.append(instance)
                    else:
                        POS = infl["pofs"]["$"]
                        if POS == "verb":
                            MOOD = infl["mood"]["$"]
                            TENSE = infl["tense"]["$"]
                            VOICE = infl["voice"]["$"]
                            try:
                                NUM = infl["num"]["$"]
                                PERS = infl["pers"]["$"]
                                posentry = POS+"-"+MOOD+"-"+NUM+"-"+PERS+"-"+TENSE+"-"+VOICE
                            except KeyError:
                                posentry = POS+"-"+MOOD+"-"+TENSE+"-"+VOICE
                        elif POS == "verb participle":
                            CASE = infl["case"]["$"]
                            GEND = infl["gend"]["$"]
                            NUM = infl["num"]["$"]                                
                            TENSE = infl["tense"]["$"]
                            VOICE = infl["voice"]["$"]  
                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM+"-"+TENSE+"-"+VOICE      
                        elif POS == "pronoun":
                            CASE = infl["case"]["$"]
                            NUM = infl["num"]["$"]
                            posentry = POS+"-"+CASE+"-"+NUM
                        elif POS == "noun":
                            try:
                                CASE = infl["case"]["$"]
                                GEND = infl["gend"]["$"]
                                NUM = infl["num"]["$"]
                                posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM    
                            except KeyError:
                                TYPE = infl["stemtype"]["$"]
                                posentry = POS+"-"+TYPE
                        elif POS == "article":
                            CASE = infl["case"]["$"]
                            GEND = infl["gend"]["$"]
                            NUM = infl["num"]["$"]
                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                        elif POS == "adjective":
                            try:
                                GEND = infl["gend"]["$"]
                                if GEND == "adverbial":
                                    posentry = POS+"-"+GEND
                                else:
                                    CASE = infl["case"]["$"]
                                    NUM = infl["num"]["$"]
                                    try:
                                        COMP = infl["comp"]["$"]
                                        posentry = POS+"-"+CASE+"-"+GEND+"-"+COMP+"-"+NUM
                                    except KeyError:
                                        posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                            except KeyError:
                                CASE = infl["case"]["$"]
                                NUM = infl["num"]["$"]
                                try:
                                    COMP = infl["comp"]["$"]
                                    posentry = POS+"-"+CASE+"-"+COMP+"-"+NUM
                                except KeyError:
                                    posentry = POS+"-"+CASE+"-"+NUM
                        else:
                            posentry = POS
                        list_pos.append(posentry)
                    list_hdwd.append(headword)
                result["Lemma"] = "|".join(list_hdwd)
                result["Morphology"] = "|".join(list_pos)
            except KeyError:
                headword = entry["rest"]["entry"]["dict"]["hdwd"]["$"]
                infl = entry["rest"]["entry"]["infl"]
                if isinstance(infl, list):
                    list_pos1 = []
                    for inst in range(len(infl)):
                        POS = infl[inst]["pofs"]["$"]
                        if POS == "verb":
                            MOOD = infl[inst]["mood"]["$"]
                            TENSE = infl[inst]["tense"]["$"]
                            VOICE = infl[inst]["voice"]["$"]
                            try:
                                NUM = infl[inst]["num"]["$"]
                                PERS = infl[inst]["pers"]["$"]
                                posentry = POS+"-"+MOOD+"-"+NUM+"-"+PERS+"-"+TENSE+"-"+VOICE
                            except KeyError:
                                posentry = POS+"-"+MOOD+"-"+TENSE+"-"+VOICE
                        elif POS == "verb participle":
                            CASE = infl[inst]["case"]["$"]
                            GEND = infl[inst]["gend"]["$"]
                            NUM = infl[inst]["num"]["$"]                                
                            TENSE = infl[inst]["tense"]["$"]
                            VOICE = infl[inst]["voice"]["$"]  
                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM+"-"+TENSE+"-"+VOICE
                        elif POS == "pronoun":
                            CASE = infl[inst]["case"]["$"]
                            NUM = infl[inst]["num"]["$"]
                            posentry = POS+"-"+CASE+"-"+NUM
                        elif POS == "noun":
                            try:
                                CASE = infl[inst]["case"]["$"]
                                GEND = infl[inst]["gend"]["$"]
                                NUM = infl[inst]["num"]["$"]
                                posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                            except KeyError:
                                TYPE = infl[inst]["stemtype"]["$"]
                                posentry = POS+"-"+TYPE
                        elif POS == "article":
                            CASE = infl[inst]["case"]["$"]
                            GEND = infl[inst]["gend"]["$"]
                            NUM = infl[inst]["num"]["$"]
                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                        elif POS == "adjective":
                            try:
                                GEND = infl[inst]["gend"]["$"]
                                if GEND == "adverbial":
                                    posentry = POS+"-"+GEND
                                else:
                                    CASE = infl[inst]["case"]["$"]
                                    NUM = infl[inst]["num"]["$"]
                                    try:
                                        COMP = infl[inst]["comp"]["$"]
                                        posentry = POS+"-"+CASE+"-"+GEND+"-"+COMP+"-"+NUM
                                    except KeyError:
                                        posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                            except KeyError:
                                CASE = infl[inst]["case"]["$"]
                                NUM = infl[inst]["num"]["$"]
                                try:
                                    COMP = infl[inst]["comp"]["$"]
                                    posentry = POS+"-"+CASE+"-"+COMP+"-"+NUM
                                except KeyError:
                                    posentry = POS+"-"+CASE+"-"+NUM
                        else:
                            posentry = POS
                        list_pos1.append(posentry)
                    posentry = "/".join(list_pos1)
                else:
                    POS = infl["pofs"]["$"]
                    if POS == "verb":
                        MOOD = infl["mood"]["$"]
                        TENSE = infl["tense"]["$"]
                        VOICE = infl["voice"]["$"]
                        try:
                            NUM = infl["num"]["$"]
                            PERS = infl["pers"]["$"]
                            posentry = POS+"-"+MOOD+"-"+NUM+"-"+PERS+"-"+TENSE+"-"+VOICE
                        except KeyError:
                            posentry = POS+"-"+MOOD+"-"+TENSE+"-"+VOICE
                    elif POS == "verb participle":
                        CASE = infl["case"]["$"]
                        GEND = infl["gend"]["$"]
                        NUM = infl["num"]["$"]                                
                        TENSE = infl["tense"]["$"]
                        VOICE = infl["voice"]["$"]  
                        posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM+"-"+TENSE+"-"+VOICE
                    elif POS == "pronoun":
                        CASE = infl["case"]["$"]
                        NUM = infl["num"]["$"]
                        posentry = POS+"-"+CASE+"-"+NUM
                    elif POS == "noun":
                        try:
                            CASE = infl["case"]["$"]
                            GEND = infl["gend"]["$"]
                            NUM = infl["num"]["$"]
                            posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM 
                        except KeyError:
                            TYPE = infl["stemtype"]["$"]
                            posentry = POS+"-"+TYPE
                    elif POS == "article":
                        CASE = infl["case"]["$"]
                        GEND = infl["gend"]["$"]
                        NUM = infl["num"]["$"]
                        posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                    elif POS == "adjective":
                        try:
                            GEND = infl["gend"]["$"]
                            if GEND == "adverbial":
                                posentry = POS+"-"+GEND
                            else:
                                CASE = infl["case"]["$"]
                                NUM = infl["num"]["$"]
                                try:
                                    COMP = infl["comp"]["$"]
                                    posentry = POS+"-"+CASE+"-"+GEND+"-"+COMP+"-"+NUM
                                except KeyError:
                                    posentry = POS+"-"+CASE+"-"+GEND+"-"+NUM
                        except KeyError:
                            CASE = infl["case"]["$"]
                            NUM = infl["num"]["$"]
                            try:
                                COMP = infl["comp"]["$"]
                                posentry = POS+"-"+CASE+"-"+COMP+"-"+NUM
                            except KeyError:
                                posentry = POS+"-"+CASE+"-"+NUM                            
                    else:
                        posentry = POS
                result["Lemma"] = headword
                result["Morphology"] = posentry
        return result

## Test with single word

In [ ]:
queryMorpheusLem("τινος")

## Test with file

Perseids Morpheus can only queried word by word. This takes quite some time... If a queried token was already queried before and therefore is already in the list, then the result is copied from list and Morpheus will not be queried for this token again. This speeds up the query. 
Also, the lemmatizer writes the results into a file `wordlemma.json`. This json-File is also used to speed up further queries.

In [ ]:
def loadingBar(count,total,size):
    percent = float(count)/float(total)*100
    sys.stdout.write("\r" + str(int(count)).rjust(3,'0')+"/"+str(int(total)).rjust(3,'0') + ' [' + '='*int(percent/10)*size + ' '*(10-int(percent/10))*size + ']')
    
word_breaks = RegexpTokenizer(r'\w+') # whitespace tokenize
data_file = open('wordlemma.json')    
lemma_data = json.load(data_file)
with open('/home/stockhausen/cltk_data/multilingual/text/patristic_text_archive_plaintext/pta0001.pta030.pta-grc1.txt', 'r') as f:
    f = f.read().lower()
    tokens = word_breaks.tokenize(f)
    lemmatized = []
    count = 0
    for token in tokens:
        count = count+1
        loadingBar(count,len(tokens),2)
        # check if queried token already in list, if yes, copy from list and don't query Morpheus for speedup
        if any(d["Word"] == token for d in lemmatized):
            matches = next(d for d in lemmatized if token == d["Word"])
            lemmatized.append(matches)
        # check if queried token already in wordlemma.json, if yes, copy from wordlemma and don't query Morpheus for speedup    
        elif any(d["Word"] == token for d in lemma_data):
            matches = next(d for d in lemma_data if token == d["Word"])
            lemmatized.append(matches)
        else:
            result = queryMorpheusLem(token)
            lemmatized.append(result)
# Delete duplicates from list and write to json file
merged_data = lemma_data + lemmatized
liste = [i for n, i in enumerate(merged_data) if i not in merged_data[n + 1:]]
with open('wordlemma.json', 'w') as fout:
    json.dump(liste, fout, ensure_ascii=False)

TODO: Manual cleanup of wordlemma.json to remove obviously wrong entries ("errors") and to choose the most probable entry in case of ambiguous results.

## View output (Lemmata)

In [ ]:
lemmata = [d['Lemma'] for d in lemmatized] # extract all lemmata from list and print
print("\n"+" ".join(lemmata))
# save as file:
# with open('/home/stockhausen/cltk_data/user_data/file_lemmatized.txt', 'w') as f:
#     f.write(" ".join(lemmata))

## Play with morphology 
TODO: tbd

In [ ]:
morphology = [d['Morphology'] for d in lemmatized]